In [52]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
from sklearn.preprocessing import  LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
pd.options.display.max_columns = 0
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [53]:
df_origin = pd.read_csv('train.csv')
final_df = pd.read_csv('df_final.csv')

In [54]:
del final_df['zipcode']

## Data Scaled

In [55]:
from sklearn.preprocessing import MinMaxScaler
features=['median_home_value','distance_to_center','time_since_last_review','host_since_day','number_of_reviews']
# Perform min-max scaling
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(final_df[features])

# Create a new DataFrame with scaled features
data_scaled = pd.DataFrame(data_scaled,columns=features)
final_df[features]=data_scaled
final_df.head()


,log_price,cleaning_fee,host_has_profile_pic,host_identity_verified,instant_bookable,number_of_reviews,thumbnail_url,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Condominium,property_type_Dorm,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_In-law,property_type_Loft,property_type_Other,property_type_Timeshare,property_type_Townhouse,property_type_Villa,accommodates_1,accommodates_2,accommodates_3,accommodates_4,accommodates_5,accommodates_6,accommodates_7,accommodates_8,accommodates_9,accommodates_10,accommodates_11,accommodates_12,accommodates_13,accommodates_14,...,bathrooms_7.5,bathrooms_8.0,check_in_24h,air_conditioning,high_end_electronics,bbq,balcony,breakfast,tv,coffee_machine,cooking_basics,white_goods,elevator,gym,child_friendly,parking,outdoor_space,host_greeting,hot_tub_sauna_or_pool,internet,long_term_stays,pets_allowed,private_entrance,secure,smoking_allowed,accessible,event_suitable,Boston,Chicago,DC,LA,NYC,SF,time_since_last_review,host_since_day,super_host,median_home_value,lat_center,long_center,distance_to_center
0,5.010635,1,1,1,0,0.184220,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0.483084,0.576242,0,0.748624,40.7200,-74.0060,0.179495
1,5.129899,1,1,0,1,0.326299,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0.401436,0.030554,0,0.860864,40.7200,-74.0060,0.243939
2,4.976734,1,1,1,1,0.402090,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0.403137,0.098230,0,0.787699,40.7200,-74.0060,0.367405
3,6.620073,1,1,1,0,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.000000,0.256710,0,0.937492,37.7749,-122.4194,0.117210
4,4.744932,1,1,1,1,0.269878,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0.447552,0.270702,0,0.633364,38.9072,-77.0369,0.145208


In [56]:
del final_df['lat_center']
del final_df['long_center']

In [57]:
X = final_df.drop(['log_price'], axis =1)
#X=final_df[important_columns]
y = final_df.log_price

In [58]:
X.shape

(73951, 107)

In [59]:
y.shape

(73951,)

In [60]:
# df_random_10_percent = final_df.sample(frac=0.10)

In [61]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=101)


## XGB Regresor 

In [62]:
from xgboost import XGBRegressor
from sklearn import metrics

xgb = XGBRegressor(objective='reg:squarederror')

xgb.fit(x_train, y_train)

y_pred_xgb = xgb.predict(x_test)

mae_xgb  = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb  = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(metrics.mean_squared_error(y_test, y_pred_xgb))
r2_xgb   = metrics.r2_score(y_test, y_pred_xgb)


print('\nMean Absolute Error of XGBoost Regressor     : ', mae_xgb)
print('\nMean Squarred Error of XGBoost Regressor     : ', mse_xgb)
print('\nRoot Mean Squarred Error of XGBoost Regressor: ', rmse_xgb)
print('\nR2 Score of XGBoost Regressor                : ', r2_xgb)


Mean Absolute Error of XGBoost Regressor     :  0.2799457246198487

Mean Squarred Error of XGBoost Regressor     :  0.144859178149419

Root Mean Squarred Error of XGBoost Regressor:  0.3806037022276833

R2 Score of XGBoost Regressor                :  0.7125310023192948


In [63]:
"""ft_weights_rf = pd.DataFrame(xgb.feature_importances_, columns=['weight'], index=x_train.columns)
ft_weights_rf.sort_values('weight', inplace=True ,ascending=False)"""
feature_importances = xgb.feature_importances_
sorted_feature_importances = sorted(zip(feature_importances, X.columns), reverse=True)
important_columns = [feature_name for importance, feature_name in sorted_feature_importances if importance > 0.003]
len(important_columns)

34

In [64]:
# Plotting feature importances
import matplotlib.pyplot as plt

plt.figure(figsize=(8,20))
plt.barh(important_columns.index, important_columns.weight, align='center') 
plt.title("Feature importances in the RandomForest model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()


AttributeError: 'list' object has no attribute 'weight'

<Figure size 800x2000 with 0 Axes>

In [ ]:
important_columns

['room_type_Entire home/apt',
 'bathrooms_1.0',
 'accommodates_16',
 'accommodates_2',
 'accommodates_1',
 'LA',
 'accommodates_3',
 'room_type_Private room',
 'bathrooms_4.5',
 'tv',
 'property_type_House',
 'bathrooms_1.5',
 'median_home_value',
 'accommodates_12',
 'distance_to_center',
 'accommodates_4',
 'accommodates_8',
 'DC',
 'property_type_Apartment',
 'bathrooms_3.5',
 'SF',
 'accommodates_10',
 'accommodates_6',
 'bathrooms_4.0',
 'gym',
 'elevator',
 'time_since_last_review',
 'bathrooms_5.5',
 'bathrooms_2.0',
 'bathrooms_5.0',
 'internet',
 'event_suitable',
 'white_goods',
 'property_type_Hostel',
 'hot_tub_sauna_or_pool',
 'property_type_Dorm',
 'NYC',
 'accommodates_14',
 'super_host',
 'accommodates_7',
 'bathrooms_3.0',
 'child_friendly',
 'property_type_Loft',
 'number_of_reviews',
 'bathrooms_0.5',
 'Chicago',
 'property_type_Other',
 'Boston',
 'room_type_Shared room',
 'property_type_Condominium',
 'property_type_Townhouse']

## XGB Tuning

In [ ]:
#df_random_10_percent = final_df.sample(frac=0.10)
X=final_df[important_columns]
y=final_df['log_price']
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=101)


In [ ]:
from xgboost import XGBRegressor
from sklearn import metrics

xgb = XGBRegressor(objective='reg:squarederror')

xgb.fit(x_train, y_train)

y_pred_xgb = xgb.predict(x_test)

mae_xgb  = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb  = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(metrics.mean_squared_error(y_test, y_pred_xgb))
r2_xgb   = metrics.r2_score(y_test, y_pred_xgb)


print('\nMean Absolute Error of XGBoost Regressor     : ', mae_xgb)
print('\nMean Squarred Error of XGBoost Regressor     : ', mse_xgb)
print('\nRoot Mean Squarred Error of XGBoost Regressor: ', rmse_xgb)
print('\nR2 Score of XGBoost Regressor                : ', r2_xgb)

In [ ]:
xgb_grid = {
     'colsample_bytree': [0.3,0.6,1], 
     'n_estimators':[1000,1200,1400],
     'max_depth': [6,8,10],
     'learning_rate': [0.005,0.01,0.05]
}
xgb = XGBRegressor()

xgb_cv = GridSearchCV(xgb, 
                      param_grid = xgb_grid, 
                      cv = 10, 
                      n_jobs = -1,
                      verbose = 2)


xgb_cv.fit(x_train, y_train)
"""xgb_grid = {
     'colsample_bytree': [0.3,0.6,1], 
     'n_estimators':[1000,1200,1400],
     'max_depth': [6,8,10],
     'learning_rate': [0.005,0.01,0.05]==========={'colsample_bytree': 0.6,
 'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 1400}
     
 """

Fitting 10 folds for each of 81 candidates, totalling 810 fits


"#  'n_estimators':[100, 200, 500, 1000],eski hali calisitrdim ve best params bu cikti:\n'colsample_bytree': 0.9,\n 'learning_rate': 0.5,\n 'max_depth': 3,\n 'n_estimators': 1000}\n en son calisan:\n {'colsample_bytree': 0.6,\n 'learning_rate': 0.1,\n 'max_depth': 4,\n 'n_estimators': 1200}\n \n xgb_grid = {\n     'colsample_bytree': [0.3,0.6,1], \n     'n_estimators':[1000,1200,1400],\n     'max_depth': [3,4,5,6],\n     'learning_rate': [0.05,0.1,0.15,0.20]in sonucu:\n     \n colsample_bytree': 0.3,\n 'learning_rate': 0.05,\n 'max_depth': 6,\n 'n_estimators': 1200}\n \n \n \n {'colsample_bytree': 0.3,\n 'learning_rate': 0.05,\n 'max_depth': 8,\n 'n_estimators': 1000}\n \n \n {'colsample_bytree': 0.3,\n 'learning_rate': 0.05,\n 'max_depth': 9,\n 'n_estimators': 600}\n "

In [ ]:
xgb_cv.best_params_

{'colsample_bytree': 0.6,
 'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 1400}

In [65]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.6, 
                         learning_rate = 0.01, 
                         max_depth = 10, 
                         n_estimators = 1400) 

xgb_tuned = xgb_tuned.fit(x_train,y_train)

In [66]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = xgb_tuned.predict(x_test)
np.sqrt(mean_squared_error(y_test, y_pred))

mae_xgb  = metrics.mean_absolute_error(y_test, y_pred)
mse_xgb  = metrics.mean_squared_error(y_test, y_pred)
rmse_xgb = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
r2_xgb   = metrics.r2_score(y_test, y_pred)


print('\nMean Absolute Error of XGBoost Regressor     : ', mae_xgb)
print('\nMean Squarred Error of XGBoost Regressor     : ', mse_xgb)
print('\nRoot Mean Squarred Error of XGBoost Regressor: ', rmse_xgb)
print('\nR2 Score of XGBoost Regressor                : ', r2_xgb)


Mean Absolute Error of XGBoost Regressor     :  0.2738234367783

Mean Squarred Error of XGBoost Regressor     :  0.13885739550723109

Root Mean Squarred Error of XGBoost Regressor:  0.3726357410491257

R2 Score of XGBoost Regressor                :  0.7244413725318579


In [ ]:
"""tum data defaulut icin
Mean Absolute Error of XGBoost Regressor     :  0.2799457246198487

Mean Squarred Error of XGBoost Regressor     :  0.144859178149419

Root Mean Squarred Error of XGBoost Regressor:  0.3806037022276833

R2 Score of XGBoost Regressor                :  0.7125310023192948


importance ile default params(000.4)

Mean Absolute Error of XGBoost Regressor     :  0.30232098831551585

Mean Squarred Error of XGBoost Regressor     :  0.17013565272101577

Root Mean Squarred Error of XGBoost Regressor:  0.4124750328456448

R2 Score of XGBoost Regressor                :  0.6623705437082166


importance ile default params(0.006)
0.69

importance ile default params(0.003)

Mean Absolute Error of XGBoost Regressor     :  0.2865505511198461

Mean Squarred Error of XGBoost Regressor     :  0.15081581432985805

Root Mean Squarred Error of XGBoost Regressor:  0.3883501182307765

R2 Score of XGBoost Regressor                :  0.7007102240005528

"""

In [ ]:
import joblib

In [ ]:
#joblib.dump(xgb_tuned,'xgb_2.joblib')

['xgb_2.joblib']

In [ ]:
import joblib
filename = 'finalized_model.sav'
joblib.dump(xgb_tuned, filename)

['finalized_model.sav']